# Comandos para baixar bibliotecas/ baixar arquivos



In [2]:
!pip install gender-guesser
!wget https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz

     |████████████████████████████████| 379 kB 16.1 MB/s 
--2022-02-07 22:58:01--  https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz
Resolving data.brasil.io (data.brasil.io)... 104.26.8.175, 172.67.71.45, 104.26.9.175, ...
Connecting to data.brasil.io (data.brasil.io)|104.26.8.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1895785 (1.8M) [application/gzip]
Saving to: ‘nomes.csv.gz’

nomes.csv.gz        100%[===================>]   1.81M  3.93MB/s    in 0.5s    

2022-02-07 22:58:02 (3.93 MB/s) - ‘nomes.csv.gz’ saved [1895785/1895785]



# Imports de Bibliotecas

In [3]:
import json
import sys
import requests
import csv
import gzip
import io
from bs4 import BeautifulSoup
import re
import tensorflow as tf
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et
from urllib.request import urlopen  
import gender_guesser.detector as gender
from unicodedata import normalize

# Classificador de Genero

In [ ]:
g = gender.Detector(case_sensitive=False)

def encode(name):
    ascii_name = normalize("NFKD", name).encode("ascii", errors="ignore").decode("ascii")
    return ascii_name.upper()

def load_data():
    fobj = io.TextIOWrapper(gzip.open("nomes.csv.gz"), encoding="utf-8")
    csv_reader = csv.DictReader(fobj)
    data = {
        row["first_name"]: row["classification"]
        for row in csv_reader
    }
    fobj.close()
    return data

name_data = load_data()

In [ ]:
def classify_download(name):
    encoded_name = encode(name)
    if encoded_name in name_data:
      return name_data[encoded_name]
    return 'A'

In [ ]:
def get_name(name):
  return name.split()[0]

def guess(name_to_gender):
  name_to_gender
  genderName = g.get_gender(name=name_to_gender)
  if (genderName) == 'male': 
    return 'M'
  else:
    if (genderName == 'female'):
        return 'F'
    else:
        genderName = classify_download(name_to_gender)
        if genderName != 'A':
          return genderName
        else:
          return  '?'

def guess_name(name):
  new_name = get_name(name)
  return guess(new_name)

Testes e exemplos de uso das funções

In [ ]:
y = get_name('Mariana Alencar')
print(y)

resp = guess('Mariana')
print(resp)

y = get_name('Kenzo Shiraishi')
print(y)
resp = guess(y)
print(resp)

print(guess_name('Lucas Kawabata'))

Mariana
F
Kenzo
M
M


# Criação dos comandos cypher

## UFAM

In [ ]:
class ResearcherUFAM(object):
    def __init__(self, name='',gender='',email=''):
        self.name = name
        self.gender = gender
        self.email = email
    def toCypher(self):
        return "CREATE(a:Author {name: '" + self.name + "', gender: '" + self.gender + "', email: '" + self.email + "'});"


In [ ]:
print("Starting scrap for UFAM")

url_ufma = 'https://icomp.ufam.edu.br/corpo-docente.html'
page_ufma = requests.get(url_ufma)
soup_ufma = BeautifulSoup(page_ufma.text, "html5lib")
ufam_professors = list()

Starting scrap for UFAM


In [ ]:
lista_ufam = soup_ufma.find('div', attrs={"class":"item-page"})
total = 0
indefinidos = 0

for campo in lista_ufam.find_all('nav'):
    contador = 0;
    # para achar o nome do professor
    for entry in campo.find_all('h2'):
        op = contador % 2
        if op == 0:
            total += 1
            name = entry.text.strip()
        contador += 1   
    email = campo.text.split("Email:")[1].strip()
    gender = guess_name(name)
    if gender == '?':
      indefinidos +=1
      print(name)
    ufam_professors.append(ResearcherUFAM(name, gender, email))

print(total)
print(indefinidos)

with open('docentes-ufam.cypher','w', encoding='ISO-8859-1') as f:
    f.write("CREATE(n:Institution {name: 'UFAM', color: '#00FFFF'});\n")
    for r in ufam_professors:
        f.write(r.toCypher()+ "\n")
        f.write("MATCH(i:Institution {name: 'UFAM'}),(a:Author {name: '"+r.name+"'}) MERGE (a)-[r:ASSOCIATED_TO]->(i);\n")

    

Altigran S. da Silva
Edjard de S. Mota
36
2


## UFMG

In [ ]:
class ResearcherUFMG(object):
    def __init__(self, name='', gender='',email=''):
        self.name = name
        self.gender = gender
        self.email = email
    def toCypher(self):
        return "CREATE(a:Author {name: '" + self.name + "', gender: '" + self.gender + "', email: '" + self.email + "'});"


In [ ]:
print("Starting scrap for UFMG")
url_ufmg = 'https://ppgcc.dcc.ufmg.br/docentes/'
page_ufmg = requests.get(url_ufmg)

Starting scrap for UFMG


In [ ]:
ufmg_professors = list()
soup_ufmg = BeautifulSoup(page_ufmg.text, 'html.parser')
table_ufmg = soup_ufmg.find('ul', attrs={"class":"professor"})

total = 0
indefinidos = 0

for campo in table_ufmg.find_all('li'):
    name = campo.find('p').text.strip()
    total += 1
    gender = guess_name(name)
    if gender == '?':
      print(name)
      indefinidos +=1
    contador = 0;
    for address in campo.find_all('a'):
        op = contador % 2
        if op == 0:
            email = address.get('href')
        contador += 1 
    ufmg_professors.append(ResearcherUFMG(name, gender, email))

print(total)
print(indefinidos)

with open('docentes-ufmg.cypher','w', encoding='ISO-8859-1') as f:
    f.write("CREATE(n:Institution {name: 'UFMG', color: '#FFFF00'});\n")
    for r in ufmg_professors:
        f.write(r.toCypher()+ "\n")
        f.write("MATCH(i:Institution {name: 'UFMG'}),(a:Author {name: '"+r.name+"'}) MERGE (a)-[r:ASSOCIATED_TO]->(i);\n")    
print("Finished !!")


Berthier Ribeiro de Araújo Neto
66
1
Finished !!


## UFRN


In [ ]:
class ResearcherUFRN(object):
    def __init__(self, name='', gender='', email=''):
        self.name = " ".join([s.capitalize() for s in name.split()])
        self.email = email
        self.gender = gender
    def toCypher(self):
        return "CREATE(a:Author {name: '" + self.name + "', gender: '" + self.gender + "', email: '" + self.email + "'});"


In [ ]:
print("Starting scrap for UFRN")
url_ufrn = 'https://sigaa.ufrn.br/sigaa/public/programa/equipe.jsf?lc=pt_BR&id=73'
page_ufrn = requests.get(url_ufrn)

Starting scrap for UFRN


In [ ]:
ufrn_professors = list()
soup_ufrn = BeautifulSoup(page_ufrn.text, 'html.parser')
table_ufrn = soup_ufrn.find(id="listagem_tabela")
contador = 0
indicador = 0

total = 0
indefinidos = 0

for campo in table_ufrn.find_all('tr'):
    if contador!=0:
      for linha in campo.find_all('td'):
          op = indicador % 6
          if op == 0:
            total += 1
            name = linha.text.strip()
            gender = guess_name(name)
            if gender == '?':
                print(name)
                indefinidos +=1
          if op == 5:
            email = linha.find('a').get('href')
            ufrn_professors.append(ResearcherUFRN(name, gender, email))
          indicador += 1
    contador += 1

print(total)
print(indefinidos)

with open("docentes-ufrn.cypher",'w', encoding='ISO-8859-1') as f:
    f.write("CREATE(n:Institution {name: 'UFRN', color: '#89F6C7'});\n")
    for p in ufrn_professors:
        f.write(p.toCypher()+ "\n")
        f.write("MATCH(i:Institution {name: 'UFRN'}),(a:Author {name: '"+p.name+"'}) MERGE (a)-[r:ASSOCIATED_TO]->(i);\n")    



LYRENE FERNANDES DA SILVA
29
1


## UNB

In [ ]:
print("Starting scraping of UNB professors")

class ResearcherUnB(object):
    def __init__(self, nome='', gender='', email='', titulo='', areas='', titulo_ano_local='', lattes=''):
        self.name = nome
        self.email = email
        self.lattesurl = lattes
        self.titulo = titulo
        self.titulo_ano_local = titulo_ano_local
        self.areas = areas
        self.gender = gender        
                
    def toCypher(self):
        return "CREATE(a:Author {name: '" + self.name + "', gender: '" + self.gender +"', lattesurl: '" + self.lattesurl + "', email: '" + self.email + "', title: '" + self.titulo+"', title_when_where: '" + self.titulo_ano_local+"', areas: '" + self.areas + "'});"



Starting scraping of UNB professors


In [ ]:
try:
    pagina_unb = requests.get("http://ppgi.unb.br/index.php?option=com_content&view=article&id=78&Itemid=471&lang=pt")
except Exception as e:
    print("Erro ao ler a pagina %s", str(e))
    sys.exit(1)

In [ ]:
pagina_unb.encoding="ISO-8859-1"
soup_unb = BeautifulSoup(pagina_unb.text, 'html.parser')
docentes_unb = list()
tabela_docentes_unb = soup_unb.find('table', attrs={"class":"docentes"})

total = 0
indefinidos = 0

contador = 0
nome = ''
email = ''
areas = ''
genero = ''
titulo = ''
titulo_ano_local = ''
lattes = ''


for campo in tabela_docentes_unb.find_all('td'):
    
    desloc = contador % 8
    if desloc == 1:
        total += 1
        nome = campo.text.split('(')[0].strip()
        genero = guess_name(nome)
        if genero == '?':
          print(nome)
          indefinidos +=1
        lattes = campo.find_all('a')[-1]['href']
    if desloc == 2:
        titulo = campo.text.split(':')[1].strip()
    if desloc == 3:
        titulo_ano_local = campo.text
    if desloc == 4:
        email = campo.text.split(':')[1].strip().replace(" [at] ", "@")
    if desloc == 5:
        areas = campo.text.split(':')[1].strip()
        docentes_unb.append(ResearcherUnB(nome, genero, email, titulo, areas, titulo_ano_local, lattes))      
    contador += 1    

print(total)
print(indefinidos)

with open("docentes-unb.cypher",'w', encoding='ISO-8859-1') as f:
    f.write("CREATE(n:Institution {name: 'UnB', color:'#E466CB'});\n")
    for d in docentes_unb:
        f.write(d.toCypher()+'\n')
        f.write("MATCH(i:Institution {name: 'UnB'}),(a:Author {name: '"+d.name+"'}) MERGE (a)-[r:ASSOCIATED_TO]->(i);\n")

print("Finished!!")


Dibio Leandro Borges
Li Weigang
31
2
Finished!!


## USP

In [ ]:
class ResearcherUSP(object):
    def __init__(self, name='', gender = '', email=''):
        self.name = name
        self.email = email
        self.gender = gender
    def toCypher(self):
        return "CREATE(a:Author {name: '" + self.name + "', gender: '" + self.gender + "'});"


In [ ]:
usp_url = 'https://www.ime.usp.br/pos-computacao/orientadores/'
uspResearchers = list()

In [ ]:
page_usp = requests.get(usp_url)

In [ ]:
page_usp.encoding="UTF-8"

soup_usp = BeautifulSoup(page_usp.text, 'html.parser')
maincolumn = soup_usp.find('figure', attrs={"class":"wp-block-table"})

total = 0
indefinidos = 0

contador = 0
for info_orientador in maincolumn.find_all('td'):
    desloc = contador % 2
    if desloc == 0:
        total += 1
        name = info_orientador.find('a').text
        gender = guess_name(name)
        if gender == '?':
          print(name)
          indefinidos +=1
        uspResearchers.append(ResearcherUSP(name, gender))
    contador += 1

print(total)
print(indefinidos)

with open('docentes-usp.cypher','w', encoding='UTF-8') as f:
    f.write("CREATE(n:Institution {name: 'USP', color: '#800000'});\n")
    for r in uspResearchers:
        f.write(r.toCypher()+ "\n")
        f.write("MATCH(i:Institution {name: 'USP'}),(a:Author {name: '"+r.name+"'}) MERGE (a)-[r:ASSOCIATED_TO]->(i);\n")
    f.close()     
print("Finished !!")


Routo Terada
40
1
Finished !!


# Buscar pesquisas cientificas


In [40]:
lista = 'list.json'

artigoNo = 0
coAutor = list()

with open(lista,'r',encoding = "UTF-8") as json_list:
  jsonData = json.load(json_list)

size = len(jsonData)

dir = '/content/curriculos/'


DictName = dict()
DictPesq = dict()
DictCoAutor = dict()

for i in range(size):
  file = dir + jsonData[i]
  tree = et.parse(file)
  name = tree.find('./DADOS-GERAIS').attrib['NOME-COMPLETO']
  otherN = tree.find('./DADOS-GERAIS').attrib['NOME-EM-CITACOES-BIBLIOGRAFICAS']
  otherNames = otherN.replace(u'\xa0', u' ').split(';')

# adicionar nomes usados para citação no dicionario associados ao autor
  for j in otherNames:
    if name in DictName:
      DictName[name].append(j)
    else:
      DictName[name]=[j]

  root = tree.getroot()

  for artigos in root.iter('ARTIGO-PUBLICADO'):
      artName = artigos.find('./DADOS-BASICOS-DO-ARTIGO').attrib['TITULO-DO-ARTIGO']
      flag = False
      
      # Verifica se a pesquisa ja foi adicionado ao dicionario
      if DictPesq:
        if artName in DictPesq:
          flag = True

      # se a pesquisa NAO está no dicionario, segue
      if not(flag):
        artigoNo +=1
        autorsList = artigos.findall('./AUTORES')
        for it in autorsList:
          autor = it.attrib["NOME-PARA-CITACAO"]
          if artName in DictPesq:
            DictPesq[artName].append(autor)
          else:
            DictPesq[artName] = [autor]





    
print(artigoNo) 
print (DictName)
print (DictPesq)
print('a')

4651
{'Adriano Alonso Veloso': ['VELOSO, A. A.', 'Veloso, Adriano', 'Veloso, A.', 'VELOSO, ADRIANO ALONSO', 'ALONSO VELOSO, ADRIANO'], 'Adriano Cesar Machado Pereira': ['PEREIRA, Adriano Cesar Machado', 'PEREIRA, Adriano', 'Pereira, Adriano C.M.', 'PEREIRA, ADRIANO C. M.', 'PEREIRA, ADRIANO CESÁR MACHADO', 'C. MACHADO PEREIRA, ADRIANO', 'Adriano C. Machado', 'MACHADO, ADRIANO C.', 'PEREIRA, ADRIANO CÉSAR MACHADO', 'MACHADO PEREIRA, A.C.', 'PEREIRA, ADRIANO C. MACHADO', 'MACHADO PEREIRA, ADRIANO CESAR', 'M. PEREIRA, ADRIANO C.'], 'Alan Mitchell Durham': ['DURHAM, A. M.', 'Durham, Alan M.', 'Durham, Alan Mitchell', 'DURHAM, ALAN', 'Durham, Alan', 'Alan Durham', 'A. Durham', 'Alan Mitchell Durham', 'Alan M. Durham', 'A. M. Durham', 'Durham, A', 'DURHAM, ALAN M'], 'Alba Cristina Magalhães Alves de Melo': ['MELO, A. C. M. A.', 'Melo, Alba C.M.A.', 'Melo, Alba Cristina Magalhaes Alves', 'DE MELO, ALBA CRISTINA MAGALHAES ALVES', 'Magalhaes Alves de Melo, Alba Cristina', 'MAGALHAES ALVES DE ME